In [164]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime, timedelta
from sklearn.preprocessing import Normalizer

In [2]:
# Read in the excel file
raw_data = pd.read_excel('Wind_data.xlsx', sheet_name='Train')

# Missing Date Handling

In [9]:
# Get the minimum and maximum time of the data
min_time = raw_data['DATETIME'].min()
max_time = raw_data['DATETIME'].max()

# Create a list for output holder
time_list = []
# Set Starting
current = min_time

# Loop through the time 
while current < max_time:
    time_list.append(current)
    current += timedelta(seconds=3600)

In [152]:
# Create the new holding dataframe for merge
full_data = pd.DataFrame({
    'DATETIME': time_list
})

# Merge the raw_data with full timeframe
full_data = full_data.merge(raw_data,how='left', on=['DATETIME'])

In [154]:
# Loop through the columns to check where is the null
for col in full_data.columns[1:]:
    # Get the numpy arrange of the column value
    work = full_data[col].values
    # Create an empty list for holding the final value
    fill = [0] * len(work)
    # Initiate the null count
    cnt = 0
    # Loop through the raw value
    for i in range(len(work)):
        # If not null 
        if not pd.isnull(work[i]):
            # And null count is 0, then we fill in the value into the new list
            if cnt == 0:
                fill[i] = work[i]
            else:
                # Else we use the starting point and ending point value to create a smoothing curve for missing value fill
                # EX: [0, NaN, 3] filled to be [0, 1.5, 3]
                if work[i] != work[i-cnt-1]:
                    fill_in_list = np.append(np.arange(work[i-cnt-1], work[i], (work[i]-work[i-cnt-1])/(cnt+1)), work[i])
                # Exception Holding for the same value case
                else:
                    fill_in_list = [work[i]] * (cnt+2)
                # Fill in the holding list with the new values
                fill[i-cnt:i+1] = fill_in_list[1:]
                # Restart the null count
                cnt = 0     
        else:
            # Count adds
            cnt+=1
    
    # Put the values back into the dataframe
    if len(work) != len(fill) and fill[-1]==0:
        full_data[col] = fill[:-1]
    else:
        full_data[col] = fill
        
    

# Normalize the columns

In [166]:
# Read in the test dataset
test_data = pd.read_excel('Wind_data.xlsx', sheet_name='Predict')

# Append the test data to the full data
train_test_df = pd.concat([full_data, test_data], ignore_index=True)

# Get the columns to be normalize
full_columns = train_test_df.columns
# Only normalize the column starts with WS
normalize_col = [x for x in full_columns if x.startswith('WS')]

# Normalize & transform the data
transformer = Normalizer().fit(train_test_df[normalize_col])
train_test_df[normalize_col] = transformer.transform(train_test_df[normalize_col])


In [173]:
# Split out the training & test dataset
normalized_train = train_test_df[~train_test_df['CF'].isnull()].copy()
normalized_test = train_test_df[train_test_df['CF'].isnull()].copy()

In [175]:
# Write the preprocessed output
normalized_train.to_csv('Prepared_Data/Normalized_Missing_Value_Filled.csv', index=False)
normalized_test.to_csv('Prepared_Data/Test_data_normalized.csv', index=False)